<h2>Installation of Packages</h2>

In [177]:
!pip install sqlalchemy
!pip install pyodbc

In [178]:
!pip install --upgrade pyodbc

In [251]:
from sqlalchemy import create_engine, MetaData, Table, inspect
import pandas as pd

pd.set_option('display.max_columns', None) # to show whole with of tables

<h2>Database Connection</h2>

In [226]:
def new_engine(dialect, server, database, user=None, password=None, integrated_security=True):
    if integrated_security:
        # For Windows authentication
        eng = f"{dialect}://{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+server"
    else:
        # For SQL Server authentication
        eng = f"{dialect}://{user}:{password}@{server}/{database}?driver=ODBC+Driver+for+SQL+Server"
    print(eng)
    return create_engine(eng)

In [227]:
# For Windows authentication
# Replace the server argument with the Server Name found when logging into SQL Management Studio
# OR
# For SQL Server authentication
# Replace the server, user and password argument with the Server Name, username and password
engine = new_engine('mssql', 'DESKTOP-PJ0B80O', 'AdventureWorks2022', integrated_security=True)

mssql://DESKTOP-PJ0B80O/AdventureWorks2022?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+server


In [228]:
print(type(engine))

<class 'sqlalchemy.engine.base.Engine'>


In [229]:
connection = engine.connect()

In [230]:
print(type(connection))

<class 'sqlalchemy.engine.base.Connection'>


<h2>Preparation of Schemas and Tables</h2>

In [231]:
inspector = inspect(engine)
schemas = inspector.get_schema_names()
print(schemas)

['db_accessadmin', 'db_backupoperator', 'db_datareader', 'db_datawriter', 'db_ddladmin', 'db_denydatareader', 'db_denydatawriter', 'db_owner', 'db_securityadmin', 'dbo', 'guest', 'HumanResources', 'INFORMATION_SCHEMA', 'Person', 'Production', 'Purchasing', 'Sales', 'sys']


<h2>Queries</h2>

In [263]:
all_queries = []
for sch in schemas:
    for table in inspector.get_table_names(schema=sch):
        if sch != 'dbo':
            col_names = ', '.join(col['name'] for col in inspector.get_columns(table_name=table, schema=sch))
            all_queries.append(f"SELECT {col_names} FROM {sch}.{table}")

qindex = 1
for query in all_queries:
    print("QUERY\n------\n", query, "\n", "----------------------------------------------", "\n")
    try:
        df_table = pd.read_sql(sql=query, con=connection)
        print(df_table.head(15))
    except Exception as e:
        print(f"Error executing query: {e}")
    print("\n"*5)
    qindex = qindex + 1

QUERY
------
 SELECT DepartmentID, Name, GroupName, ModifiedDate FROM HumanResources.Department 
 ---------------------------------------------- 

    DepartmentID                        Name  \
0              1                 Engineering   
1              2                 Tool Design   
2              3                       Sales   
3              4                   Marketing   
4              5                  Purchasing   
5              6    Research and Development   
6              7                  Production   
7              8          Production Control   
8              9             Human Resources   
9             10                     Finance   
10            11        Information Services   
11            12            Document Control   
12            13           Quality Assurance   
13            14  Facilities and Maintenance   
14            15      Shipping and Receiving   

                               GroupName ModifiedDate  
0               Research and

    BusinessEntityID                               rowguid  \
0                  1  0C7D8F81-D7B1-4CF0-9C0A-4CD8B6B50087   
1                  2  6648747F-7843-4002-B317-65389684C398   
2                  3  568204DA-93D7-42F4-8A7A-4446A144277D   
3                  4  0EFF57B9-4F4F-41A6-8867-658C199A5FC0   
4                  5  B82F88D1-FF79-4FD9-8C54-9D24C140F647   
5                  6  1B3D077A-1941-4D6E-8328-F7DC03595565   
6                  7  C1898370-A36F-43A2-987C-0BF24FE3FB82   
7                  8  2B50ABB8-ABAB-412B-A4D0-4FD5EBEB5CBE   
8                  9  5C0AB449-A087-4D8D-834F-3726061B6BFA   
9                 10  0F3CC1D7-F484-4BDE-B088-B11EF03E2F52   
10                11  A417A3D1-00EB-4D7F-B793-F93DC2C5391D   
11                12  EBD8A50F-322E-4426-A39A-566FD5535B1C   
12                13  1D61C426-FF79-4E23-9EA5-9610CB063F3B   
13                14  C596EC29-BA16-441B-A5EC-7FED4C8344C4   
14                15  955E421A-0AEB-4D8F-AE74-88EFFB82E41A   

       

    BusinessEntityID                                  PasswordHash  \
0                  1  pbFwXWE99vobT6g+vPWFy93NtUU/orrIWafF01hccfM=   
1                  2  bawRVNrZQYQ05qF05Gz6VLilnviZmrqBReTTAGAudm0=   
2                  3  8BUXrZfDqO1IyHCWOYzYmqN1IhTUn3CJMpdx/UCQ3iY=   
3                  4  SjLXpiarHSlz+6AG+H+4QpB/IPRzras/+9q/5Wr7tf8=   
4                  5  8FYdAiY6gWuBsgjCFdg0UibtsqOcWHf9TyaHIP7+paA=   
5                  6  u5kbN5n84NRE1h/a+ktdRrXucjgrmfF6wZC4g82rjHM=   
6                  7  zSqerln8T8eq3nYHC4Lx4vMuxZaxkDylVwWnP2ZT6QA=   
7                  8  s+FUWADIZzXBKpcbxe4OwL2uiJmjLogJNYXXHvc1X/k=   
8                  9  fCvCTy3RwzA2LNhhhYUbT7erkb9Au5wyM2q7ReHroV0=   
9                 10  /8biMrxuAtETGeIuloSrMQHBraZtZ+eU2z5OJ1Fhn6M=   
10                11  iaZ6ky76dbOG+0Y069v4bm78UfhfGXSeYtxp4Vgd15o=   
11                12  I9HGCr3jbwF3LYBlVsM/cOC2IHfg7ns5t2xejnWZ9Ko=   
12                13  3ZuoojogvvBKmtr+iHeqWoiZNCzp6N1abPoymjp+O+4=   
13                14

    ProductID                   Name ProductNumber  MakeFlag  \
0           1        Adjustable Race       AR-5381     False   
1           2           Bearing Ball       BA-8327     False   
2           3        BB Ball Bearing       BE-2349      True   
3           4  Headset Ball Bearings       BE-2908     False   
4         316                  Blade       BL-2036      True   
5         317            LL Crankarm       CA-5965     False   
6         318            ML Crankarm       CA-6738     False   
7         319            HL Crankarm       CA-7457     False   
8         320        Chainring Bolts       CB-2903     False   
9         321          Chainring Nut       CN-6137     False   
10        322              Chainring       CR-7833     False   
11        323             Crown Race       CR-9981     False   
12        324            Chain Stays       CS-2812      True   
13        325                Decal 1       DC-8732     False   
14        326                Decal 2    

    ProductModelID  ProductDescriptionID CultureID ModifiedDate
0                1                  1199    en       2013-04-30
1                1                  1467    ar       2013-04-30
2                1                  1589    fr       2013-04-30
3                1                  1712    th       2013-04-30
4                1                  1838    he       2013-04-30
5                1                  1965    zh-cht   2013-04-30
6                2                  1210    en       2013-04-30
7                2                  1476    ar       2013-04-30
8                2                  1598    fr       2013-04-30
9                2                  1721    th       2013-04-30
10               2                  1847    he       2013-04-30
11               2                  1974    zh-cht   2013-04-30
12               3                  1195    en       2013-04-30
13               3                  1464    ar       2013-04-30
14               3                  1586

...

    WorkOrderID  ProductID  OperationSequence  LocationID ScheduledStartDate  \
0            13        747                  1          10         2011-06-03   
1            13        747                  2          20         2011-06-03   
2            13        747                  3          30         2011-06-03   
3            13        747                  4          40         2011-06-03   
4            13        747                  6          50         2011-06-03   
5            13        747                  7          60         2011-06-03   
6            14        748                  1          10         2011-06-03   
7            14        748                  2          20         2011-06-03   
8            14        748                  3          30         2011-06-03   
9            14        748                  5          45         2011-06-03   
10           14        748                  6          50         2011-06-03   
11           14        748              

   ShipMethodID                Name  ShipBase  ShipRate  \
0             1  XRQ - TRUCK GROUND      3.95      0.99   
1             2        ZY - EXPRESS      9.95      1.99   
2             3   OVERSEAS - DELUXE     29.95      2.99   
3             4    OVERNIGHT J-FAST     21.95      1.29   
4             5   CARGO TRANSPORT 5      8.99      1.49   

                                rowguid ModifiedDate  
0  6BE756D9-D7BE-4463-8F2C-AE60C710D606   2008-04-30  
1  3455079B-F773-4DC6-8F1E-2A58649C4AB8   2008-04-30  
2  22F4E461-28CF-4ACE-A980-F686CF112EC8   2008-04-30  
3  107E8356-E7A8-463D-B60C-079FFF467F3F   2008-04-30  
4  B166019A-B134-4E76-B957-2B0490C610ED   2008-04-30  






QUERY
------
 SELECT BusinessEntityID, AccountNumber, Name, CreditRating, PreferredVendorStatus, ActiveFlag, PurchasingWebServiceURL, ModifiedDate FROM Purchasing.Vendor 
 ---------------------------------------------- 

    BusinessEntityID AccountNumber                        Name  CreditRating  \
0       

    CustomerID  PersonID  StoreID  TerritoryID AccountNumber  \
0            1       NaN    934.0            1    AW00000001   
1            2       NaN   1028.0            1    AW00000002   
2            3       NaN    642.0            4    AW00000003   
3            4       NaN    932.0            4    AW00000004   
4            5       NaN   1026.0            4    AW00000005   
5            6       NaN    644.0            4    AW00000006   
6            7       NaN    930.0            1    AW00000007   
7            8       NaN   1024.0            5    AW00000008   
8            9       NaN    620.0            5    AW00000009   
9           10       NaN    928.0            6    AW00000010   
10          11       NaN   1022.0            6    AW00000011   
11          12       NaN    622.0            6    AW00000012   
12          13       NaN    434.0            7    AW00000013   
13          14       NaN   1020.0            8    AW00000014   
14          15       NaN    624.0       

    SalesOrderID  SalesReasonID ModifiedDate
0          43697              5   2011-05-31
1          43697              9   2011-05-31
2          43702              5   2011-06-01
3          43702              9   2011-06-01
4          43703              5   2011-06-01
5          43703              9   2011-06-01
6          43706              5   2011-06-02
7          43706              9   2011-06-02
8          43707              5   2011-06-02
9          43707              9   2011-06-02
10         43709              5   2011-06-02
11         43709              9   2011-06-02
12         43710              5   2011-06-02
13         43710              9   2011-06-02
14         43711              5   2011-06-03






QUERY
------
 SELECT BusinessEntityID, TerritoryID, SalesQuota, Bonus, CommissionPct, SalesYTD, SalesLastYear, rowguid, ModifiedDate FROM Sales.SalesPerson 
 ---------------------------------------------- 

    BusinessEntityID  TerritoryID  SalesQuota   Bonus  CommissionPc

    SpecialOfferID                  Description  DiscountPct  \
0                1                  No Discount         0.00   
1                2     Volume Discount 11 to 14         0.02   
2                3     Volume Discount 15 to 24         0.05   
3                4     Volume Discount 25 to 40         0.10   
4                5     Volume Discount 41 to 60         0.15   
5                6      Volume Discount over 60         0.20   
6                7  Mountain-100 Clearance Sale         0.35   
7                8   Sport Helmet Discount-2002         0.10   
8                9           Road-650 Overstock         0.30   
9               10           Mountain Tire Sale         0.50   
10              11   Sport Helmet Discount-2003         0.15   
11              12           LL Road Frame Sale         0.35   
12              13       Touring-3000 Promotion         0.15   
13              14       Touring-1000 Promotion         0.20   
14              15        Half-Price Ped

<h3>We see that the schemas Person and Sales contain all customer related information, which may have something useful to tell us about the customers.</h3>

In [264]:
all_queries = []
for sch in schemas:
    for table in inspector.get_table_names(schema=sch):
        if sch == 'Person' or sch == 'Sales':
            col_names = ', '.join(col['name'] for col in inspector.get_columns(table_name=table, schema=sch))
            all_queries.append(f"SELECT {col_names} FROM {sch}.{table}")

qindex = 1
for query in all_queries:
    print("QUERY", qindex, "\n------\n", query, "\n", "----------------------------------------------", "\n")
    try:
        df_table = pd.read_sql(sql=query, con=connection)
        print(df_table.head(15))
    except Exception as e:
        print(f"Error executing query number {qindex}: {e}")
    print("\n"*4)
    qindex = qindex + 1


QUERY 1 
------
 SELECT AddressID, AddressLine1, AddressLine2, City, StateProvinceID, PostalCode, SpatialLocation, rowguid, ModifiedDate FROM Person.Address 
 ---------------------------------------------- 

Error executing query number 1: (pyodbc.ProgrammingError) ('ODBC SQL type -151 is not yet supported.  column-index=6  type=-151', 'HY106')
(Background on this error at: https://sqlalche.me/e/14/f405)





QUERY 2 
------
 SELECT AddressTypeID, Name, rowguid, ModifiedDate FROM Person.AddressType 
 ---------------------------------------------- 

   AddressTypeID         Name                               rowguid  \
0              1      Billing  B84F78B1-4EFE-4A0E-8CB7-70E9F112F886   
1              2         Home  41BC2FF6-F0FC-475F-8EB9-CEC0805AA0F2   
2              3  Main Office  8EEEC28C-07A2-4FB9-AD0A-42D4A0BBC575   
3              4      Primary  24CB3088-4345-47C4-86C5-17B535133D1E   
4              5     Shipping  B29DA3F8-19A3-47DA-9DAA-15C84F4A83A5   
5              6   

    BusinessEntityID  EmailAddressID                  EmailAddress  \
0                  1               1      ken0@adventure-works.com   
1                  2               2    terri0@adventure-works.com   
2                  3               3  roberto0@adventure-works.com   
3                  4               4      rob0@adventure-works.com   
4                  5               5     gail0@adventure-works.com   
5                  6               6   jossef0@adventure-works.com   
6                  7               7    dylan0@adventure-works.com   
7                  8               8    diane1@adventure-works.com   
8                  9               9     gigi0@adventure-works.com   
9                 10              10  michael6@adventure-works.com   
10                11              11   ovidiu0@adventure-works.com   
11                12              12  thierry0@adventure-works.com   
12                13              13   janice0@adventure-works.com   
13                14

   PhoneNumberTypeID  Name            ModifiedDate
0                  1  Cell 2017-12-13 13:19:22.273
1                  2  Home 2017-12-13 13:19:22.273
2                  3  Work 2017-12-13 13:19:22.273





QUERY 13 
------
 SELECT StateProvinceID, StateProvinceCode, CountryRegionCode, IsOnlyStateProvinceFlag, Name, TerritoryID, rowguid, ModifiedDate FROM Person.StateProvince 
 ---------------------------------------------- 

    StateProvinceID StateProvinceCode CountryRegionCode  \
0                 1               AB                 CA   
1                 2               AK                 US   
2                 3               AL                 US   
3                 4               AR                 US   
4                 5               AS                 AS   
5                 6               AZ                 US   
6                 7               BC                 CA   
7                 8               BY                 DE   
8                 9               CA 

    CustomerID  PersonID  StoreID  TerritoryID AccountNumber  \
0            1       NaN    934.0            1    AW00000001   
1            2       NaN   1028.0            1    AW00000002   
2            3       NaN    642.0            4    AW00000003   
3            4       NaN    932.0            4    AW00000004   
4            5       NaN   1026.0            4    AW00000005   
5            6       NaN    644.0            4    AW00000006   
6            7       NaN    930.0            1    AW00000007   
7            8       NaN   1024.0            5    AW00000008   
8            9       NaN    620.0            5    AW00000009   
9           10       NaN    928.0            6    AW00000010   
10          11       NaN   1022.0            6    AW00000011   
11          12       NaN    622.0            6    AW00000012   
12          13       NaN    434.0            7    AW00000013   
13          14       NaN   1020.0            8    AW00000014   
14          15       NaN    624.0       

    BusinessEntityID                                 Name  SalesPersonID  \
0                292                 Next-Door Bike Store            279   
1                294       Professional Sales and Service            276   
2                296                       Riders Company            277   
3                298                   The Bike Mechanics            275   
4                300                    Nationwide Supply            286   
5                302                Area Bike Accessories            281   
6                304         Bicycle Accessories and Kits            283   
7                306                Clamps & Brackets Co.            275   
8                308           Valley Bicycle Specialists            277   
9                310                    New Bikes Company            279   
10               312  Vinyl and Plastic Goods Corporation            282   
11               314                Top of the Line Bikes            288   
12          

<p><h4>We see that the tables 1, 4, 7, 10, 13, 15, 16, 20 (OrderQty), 21 (TerritoryID, SubTotal), and 27 (City) may be able to provide us the information we are looking for.</h4></p>

<p><h4>We also take a look the <em>views</em> that are already defined in the original database</h4></p>

In [267]:
all_queries = []
for sch in schemas:
    for view in inspector.get_view_names(schema=sch):
        if sch == 'Person' or sch == 'Sales':
            col_names = ', '.join(col['name'] for col in inspector.get_columns(table_name=view, schema=sch))
            all_queries.append(f"SELECT {col_names} FROM {sch}.{view}")

qindex = 1
for query in all_queries:
    print("QUERY", qindex, "\n------\n", query, "\n", "----------------------------------------------", "\n")
    try:
        df_view = pd.read_sql(sql=query, con=connection)
        print(df_view.head(15))
    except Exception as e:
        print(f"Error executing query number {qindex}: {e}")
    print("\n"*4)
    qindex = qindex + 1

QUERY 1 
------
 SELECT BusinessEntityID, FirstName, MiddleName, LastName, TelephoneNumber, TelephoneSpecialInstructions, Street, City, StateProvince, PostalCode, CountryRegion, HomeAddressSpecialInstructions, EMailAddress, EMailSpecialInstructions, EMailTelephoneNumber, rowguid, ModifiedDate FROM Person.vAdditionalContactInfo 
 ---------------------------------------------- 

   BusinessEntityID  FirstName MiddleName     LastName TelephoneNumber  \
0               291    Gustavo       None       Achong    425-555-1112   
1               293  Catherine         R.         Abel    206-555-2222   
2               295        Kim       None  Abercrombie    605-555-9877   
3               297   Humberto       None      Acevedo            None   
4               299      Pilar       None     Ackerman            None   
5               301    Frances         B.        Adams            None   
6               303   Margaret         J.        Smith    206-555-2222   
7               305      Car

    BusinessEntityID  TotalPurchaseYTD DateFirstPurchase BirthDate  \
0                 18               0.0               NaT       NaT   
1                 19               0.0               NaT       NaT   
2                 20               0.0               NaT       NaT   
3                 21               0.0               NaT       NaT   
4                 22               0.0               NaT       NaT   
5                 23               0.0               NaT       NaT   
6                 24               0.0               NaT       NaT   
7                 25               0.0               NaT       NaT   
8                 26               0.0               NaT       NaT   
9                 27               0.0               NaT       NaT   
10                28               0.0               NaT       NaT   
11                29               0.0               NaT       NaT   
12                30               0.0               NaT       NaT   
13                31

    BusinessEntityID                          Name  AnnualSales  \
0                328                 Purchase Mart    1500000.0   
1                330         Major Sport Suppliers    3000000.0   
2                332   Family's Favorite Bike Shop     800000.0   
3                334                  Global Plaza     800000.0   
4                336  Imported and Domestic Cycles    1000000.0   
5                338              Systematic Sales    3000000.0   
6                340               eCommerce Bikes    3000000.0   
7                342            Mountain Toy Store     800000.0   
8                344      Retail Sales and Service     800000.0   
9                346       Designated Distributors    3000000.0   
10               348         Bold Bike Accessories    3000000.0   
11               350                   Twin Cycles     800000.0   
12               352            Tiny Bike Boutique     300000.0   
13               354     Acclaimed Bicycle Company    3000000.

In [485]:
import pandas as pd

# Choose schemas
schemas_of_interest = ['Person', 'Sales']

table_sizes = {}

# Iterate over schemas and tables
for sch in schemas_of_interest:
    table_sizes[sch] = {}
    for view in inspector.get_view_names(schema=sch):
        tables = f'''
            SELECT COUNT(*) FROM {sch}.{view}
        '''
        # Retrieve table sizes and save results in a DataFrame
        df1 = pd.read_sql(sql=tables, con=connection)
        
        # Convert DataFrame to dictionary and store in the main dictionary
        nbr_rows = df1.iloc[0, 0]
        table_sizes[sch][view] = {'rows': nbr_rows}

        columns = f'''
            SELECT COLUMN_NAME
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_SCHEMA = '{sch}' AND TABLE_NAME = '{view}';
        '''

        # Retrieve columns and save results in a DataFrame
        df2 = pd.read_sql(sql=columns, con=connection)
        nbr_cols = df2.shape[0]
        column_list = [df2.iloc[index, 0] for index in range(nbr_cols)]
        
        # Save the sizes of each view and retrieve the names of the columns
        table_sizes[sch][view]['columns'] = column_list
        
# Print the dictionary
print("Table Sizes:", table_sizes)

Table Sizes: {'Person': {'vAdditionalContactInfo': {'rows': 10, 'columns': ['BusinessEntityID', 'FirstName', 'MiddleName', 'LastName', 'TelephoneNumber', 'TelephoneSpecialInstructions', 'Street', 'City', 'StateProvince', 'PostalCode', 'CountryRegion', 'HomeAddressSpecialInstructions', 'EMailAddress', 'EMailSpecialInstructions', 'EMailTelephoneNumber', 'rowguid', 'ModifiedDate']}, 'vStateProvinceCountryRegion': {'rows': 181, 'columns': ['StateProvinceID', 'StateProvinceCode', 'IsOnlyStateProvinceFlag', 'StateProvinceName', 'TerritoryID', 'CountryRegionCode', 'CountryRegionName']}}, 'Sales': {'vIndividualCustomer': {'rows': 18508, 'columns': ['BusinessEntityID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix', 'PhoneNumber', 'PhoneNumberType', 'EmailAddress', 'EmailPromotion', 'AddressType', 'AddressLine1', 'AddressLine2', 'City', 'StateProvinceName', 'PostalCode', 'CountryRegionName', 'Demographics']}, 'vPersonDemographics': {'rows': 19972, 'columns': ['BusinessEntityID', 'Tota

In [486]:
for schema, views in table_sizes.items():
    print(f"Schema: {schema}")
    for view, properties in views.items():
        print(f"\n       View: {view}")
        print(f"            Rows: {properties['rows']}")
        print(f"Columns: \n{', '.join(properties['columns'])}")

Schema: Person

       View: vAdditionalContactInfo
            Rows: 10
Columns: 
BusinessEntityID, FirstName, MiddleName, LastName, TelephoneNumber, TelephoneSpecialInstructions, Street, City, StateProvince, PostalCode, CountryRegion, HomeAddressSpecialInstructions, EMailAddress, EMailSpecialInstructions, EMailTelephoneNumber, rowguid, ModifiedDate

       View: vStateProvinceCountryRegion
            Rows: 181
Columns: 
StateProvinceID, StateProvinceCode, IsOnlyStateProvinceFlag, StateProvinceName, TerritoryID, CountryRegionCode, CountryRegionName
Schema: Sales

       View: vIndividualCustomer
            Rows: 18508
Columns: 
BusinessEntityID, Title, FirstName, MiddleName, LastName, Suffix, PhoneNumber, PhoneNumberType, EmailAddress, EmailPromotion, AddressType, AddressLine1, AddressLine2, City, StateProvinceName, PostalCode, CountryRegionName, Demographics

       View: vPersonDemographics
            Rows: 19972
Columns: 
BusinessEntityID, TotalPurchaseYTD, DateFirstPurchase, Bi

<h4>From the results above, we can notice that for a more accurate comparison between male and female buyers, we'll need large samples. We choose thus to continue with the tables <em>vIndividualCustomer</em> and <em>vPersonDemographics</em> from the schema <em>Person</em></h4>

In [497]:
joint_table = f'''
                SELECT *
                FROM Sales.vPersonDemographics AS VPD
                JOIN Sales.vIndividualCustomer AS VIC
                ON VIC.BusinessEntityID = VPD.BusinessEntityID;
            '''
final_df = pd.read_sql(sql=joint_table, con=connection)
print(final_df.head(5))

   BusinessEntityID  TotalPurchaseYTD DateFirstPurchase  BirthDate  \
0              9196           -13.500        2003-02-06 1963-06-14   
1             17400           -25.000        2004-01-15 1941-01-05   
2              5124          1130.990        2003-12-20 1979-04-28   
3             17691          1354.545        2002-12-11 1943-02-26   
4              4073           -61.000        2003-12-10 1958-07-01   

  MaritalStatus         YearlyIncome Gender  TotalChildren  \
0             M          50001-75000      M              4   
1             S          25001-50000      F              4   
2             S          25001-50000      M              0   
3             S  greater than 100000      M              2   
4             M         75001-100000      M              5   

   NumberChildrenAtHome        Education      Occupation  HomeOwnerFlag  \
0                     2       Bachelors     Professional           True   
1                     0       Bachelors       Management

In [498]:
correlation_matrix = final_df[['YearlyIncome', 'TotalPurchaseYTD', 'NumberCarsOwned']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap')
plt.show()

ValueError: could not convert string to float: '50001-75000'

In [505]:
final_df['YearlyIncome_x'] = final_df['YearlyIncome'].str.split('- ', expand=True)[-1]

KeyError: -1